In [1]:
import tensorflow as tf
import os
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

In [2]:
#处理输入的影评，每一行是一些词
#x = [N,max_len,300]
neg_dir = r'rt-polaritydata/rt-polarity.neg'
pos_dir = r'rt-polaritydata/rt-polarity.pos'
with open(neg_dir,'rb') as f:
    data = f.read().split('\n')
    neg_words = [0]*len(data)
    for d in range(len(data)):
        neg_words[d] = data[d].split(' ')
print(neg_words[1])
max_len = 0
for d in neg_words:
    if len(d)> max_len:
        max_len = len(d)
print(max_len)
with open(pos_dir,'rb') as f:
    data = f.read().split('\n')
    pos_words = [0]*len(data)
    for d in range(len(data)):
        pos_words[d] = data[d].split(' ')
print(pos_words[1])     
for d in pos_words:
    if len(d)> max_len:
        max_len = len(d)      
#获取x的axis=1        
print(max_len)

["it's", 'so', 'laddish', 'and', 'juvenile', ',', 'only', 'teenage', 'boys', 'could', 'possibly', 'find', 'it', 'funny', '.', '']
57
['the', 'gorgeously', 'elaborate', 'continuation', 'of', '"', 'the', 'lord', 'of', 'the', 'rings', '"', 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'cannot', 'adequately', 'describe', 'co-writer/director', 'peter', "jackson's", 'expanded', 'vision', 'of', 'j', '.', 'r', '.', 'r', '.', "tolkien's", 'middle-earth', '.', '']
61


In [3]:
#word_to_vector
vectors_dir = r'rt-polaritydata/test2.w2v'
with open(vectors_dir,'rb') as f:
    data = f.read()
    data = str(data)
    data = data.split('\n')
    i=0
    word_to_vec = {}
    vec = []
    print(len(data))
    for d in range(len(data)):
        if i:
            dd = data[d].split(' ')
            word = dd[0]
            vecs = dd[1:]
            vec = []
            for v in vecs:
                if v and word is not '':
                    vec.append(float(v))
            word_to_vec[word] = vec  
        i += 1
print(len(word_to_vec))

21428
21427


In [4]:
#text word to vector
worddim=300
null_fill = [0.0]*worddim
x_neg = np.zeros((len(neg_words),max_len,worddim))
print(x_neg.shape)
for line in range(len(neg_words)):
    for word_ind in range(max_len):
        if word_ind >= len(neg_words[line]):
            x_neg[line][word_ind] = np.array(null_fill)
        else:
            if neg_words[line][word_ind] in word_to_vec and word_to_vec[neg_words[line][word_ind]]:
                x_neg[line][word_ind] = np.array(word_to_vec[neg_words[line][word_ind]])
            else:
                x_neg[line][word_ind] = np.array(null_fill)


null_fill = [0.0]*worddim
x_pos = np.zeros((len(pos_words),max_len,worddim))
print(x_pos.shape)
for line in range(len(pos_words)):
    for word_ind in range(max_len):
        if word_ind >= len(pos_words[line]):
            x_pos[line][word_ind] = np.array(null_fill)
        else:
            if pos_words[line][word_ind] in word_to_vec and word_to_vec[pos_words[line][word_ind]]:
                x_pos[line][word_ind] = np.array(word_to_vec[pos_words[line][word_ind]])
            else:
                x_pos[line][word_ind] = np.array(null_fill)


(5332, 61, 300)
(5332, 61, 300)


In [5]:
#x of shape(14012, 447, 300) and y of shape(14012,)
from sklearn.preprocessing import OneHotEncoder
y_neg = np.zeros((len(neg_words)))
y_pos = np.ones((len(pos_words)))
#y_ = np.concatenate((y_neg,y_pos),axis=0)
#x_ = np.concatenate((x_neg,x_pos),axis=0)
train_data = np.concatenate((x_neg[len(neg_words)/10:],x_pos[len(neg_words)/10:]),axis=0)
train_label = np.concatenate((y_neg[len(neg_words)/10:],y_pos[len(neg_words)/10:]),axis=0)

ohe = OneHotEncoder()
ohe.fit([[0],[1]])
train_label = np.array(ohe.transform(np.transpose([train_label,])).toarray()) #trainsform into onehot label

np.random.seed(231)
np.random.shuffle(train_data)
np.random.seed(231)
np.random.shuffle(train_label)

test_data = np.concatenate((x_neg[:len(neg_words)/10],x_pos[:len(pos_words)/10]),axis=0)
test_label = np.concatenate((y_neg[:len(neg_words)/10],y_pos[:len(pos_words)/10]),axis=0)
test_label = np.array(ohe.transform(np.transpose([test_label,])).toarray()) #trainsform into onehot label
np.random.seed(131)
np.random.shuffle(test_data)
np.random.seed(131)
np.random.shuffle(test_label)


In [ ]:
from math import sqrt
import random

print("trainsamples:",np.sum(train_label,axis=0))
print("testsamples:",np.sum(test_label,axis=0))
w2v_dim=300
conv_filter=100
batch_size =64
batch_size_test=128
drop_rate=0.5
train_size=train_data.shape[0]
test_size=test_data.shape[0]
epochs=50


print("building the model..")
xs = tf.placeholder(tf.float32,[None,max_len,w2v_dim])
ys = tf.placeholder(tf.float32,[None,2])


'''
your model
'''

print("training...")
for e in range(epochs):
    seed=random.randint(0,99)
    print("seed:",seed)
    np.random.seed(seed)
    np.random.shuffle(train_data)
    np.random.seed(seed)
    np.random.shuffle(train_label)


    loss_array=[]
    train_acc=[]
    print("epoch:",e)
    # your training processs


('trainsamples:', array([ 4799.,  4799.]))
('testsamples:', array([ 533.,  533.]))
building the model..
Tensor("conv1d/BiasAdd:0", shape=(?, 59, 100), dtype=float32)
Tensor("conv1d_2/BiasAdd:0", shape=(?, 58, 100), dtype=float32)
Tensor("conv1d_3/BiasAdd:0", shape=(?, 57, 100), dtype=float32)
training...
('seed:', 19)
('epoch:', 0)
('loss:', 1.7373345, '   r:', 0.77993172)
('loss:', 2.1535962, '   r:', 0.75690371)
('loss:', 1.709811, '   r:', 0.74079651)
('valid_accuracy: ', 0.7373046875, '  train_accuracy: ', 0.73657718120805371, '  training_loss: ', 0.78570689770999369)
('seed:', 59)
('epoch:', 1)
('loss:', 0.53834671, '   r:', 0.022515146)
('loss:', 0.49267891, '   r:', 0.022731388)
('loss:', 0.57587916, '   r:', 0.023035405)
('valid_accuracy: ', 0.744140625, '  train_accuracy: ', 0.79781879194630867, '  training_loss: ', 0.52559841079199876)
('seed:', 52)
('epoch:', 2)
('loss:', 0.41638356, '   r:', 0.027635679)
('loss:', 0.3812924, '   r:', 0.02787937)
('loss:', 0.45667437, '   r:

('loss:', 0.08381591, '   r:', 0.048146002)
('loss:', 0.093176708, '   r:', 0.048150089)
('valid_accuracy: ', 0.7734375, '  train_accuracy: ', 0.99863674496644295, '  training_loss: ', 0.10446011460067442)
('seed:', 21)
('epoch:', 29)
('loss:', 0.11062037, '   r:', 0.055991523)
('loss:', 0.10535122, '   r:', 0.055668451)
('loss:', 0.10081485, '   r:', 0.055289708)
('valid_accuracy: ', 0.7587890625, '  train_accuracy: ', 0.99853187919463082, '  training_loss: ', 0.10193302487366951)
('seed:', 59)
('epoch:', 30)
('loss:', 0.075239792, '   r:', 0.047335468)
('loss:', 0.091090143, '   r:', 0.04747206)
('loss:', 0.12042321, '   r:', 0.047603711)
('valid_accuracy: ', 0.76171875, '  train_accuracy: ', 0.99884647651006708, '  training_loss: ', 0.10105900476442888)
('seed:', 34)
('epoch:', 31)
('loss:', 0.11238681, '   r:', 0.065886796)
('loss:', 0.087750711, '   r:', 0.065753877)
('loss:', 0.098862469, '   r:', 0.065596946)
('valid_accuracy: ', 0.76171875, '  train_accuracy: ', 0.9984270134228